In [1]:
#Import files
import datetime
import pandas as pd
from calendar import monthrange
import numpy as np
from pandas.tseries.offsets import MonthEnd
import csv
import os
import warnings; warnings.filterwarnings('ignore')
import pantab
from tableauhyperapi import TableName
from glob import glob
import sqlite3
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine

In [2]:
#Path
os.chdir('D:\\Survey Data\\Facebook')

In [3]:
All_QH = pd.read_csv("1_CookieCutter\\Data\\All_QH.csv",encoding='cp1252')

In [4]:
All_Code = pd.read_csv("1_CookieCutter\\Data\\All_DataNumbers.csv",encoding='cp1252')

In [5]:
All_Label = pd.read_csv("1_CookieCutter\\Data\\All_DataLabels.csv",encoding='cp1252')

# Single to Multi function

In [6]:
def SingletoMulti(singleqid,multiqid,text,code):
    All_Code[multiqid] = np.where((All_Code[singleqid]==code),1,0)
    All_Label[multiqid] = np.where((All_Code[singleqid]==code),text,0)

# Concept A

In [7]:
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_1_ConceptA','Facebook',1)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_2_ConceptA','Google',2)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_3_ConceptA','Amazon',3)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_4_ConceptA','Yahoo!',4)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_5_ConceptA','Twitter',5)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_6_ConceptA','LinkedIn',6)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_7_ConceptA','TikTok',7)
SingletoMulti('BRAND_ATTRIBUTION_Q10_Lr1','Q10_8_ConceptA','Pinterest',8)

In [8]:
Sec1 = ['R14r1','R14r2','R14r3','R14r4','R14r5','R14r6','R14r7','R14r8','R14r9','R14r98','R14r99','PRE_FAVORABILITY_Q0r1','PRE_FAVORABILITY_Q0r2','PRE_FAVORABILITY_Q0r3','PRE_FAVORABILITY_Q0r4','LIKEABILITY_Q1_Lr1','ACTION_Q4_Lr1r1','ACTION_Q4_Lr1r2','ACTION_Q4_Lr1r3','ACTION_Q4_Lr1r4','ACTION_Q4_Lr1r5','ACTION_Q4_Lr1r6','ACTION_Q4_Lr1r7','CLARITY_Q5_Lr1','RELEVANCE_Q6_Lr1','DIFFERENTIATION_Q7_Lr1','EMOTIONS_Q9_Lr1r1','EMOTIONS_Q9_Lr1r2','EMOTIONS_Q9_Lr1r3','EMOTIONS_Q9_Lr1r4','EMOTIONS_Q9_Lr1r5','EMOTIONS_Q9_Lr1r6','EMOTIONS_Q9_Lr1r7','EMOTIONS_Q9_Lr1r8','EMOTIONS_Q9_Lr1r9','EMOTIONS_Q9_Lr1r10','EMOTIONS_Q9_Lr1r11','EMOTIONS_Q9_Lr1r12','EMOTIONS_Q9_Lr1r13','EMOTIONS_Q9_Lr1r14','EMOTIONS_Q9_Lr1r15','EMOTIONS_Q9_Lr1r16','EMOTIONS_Q9_Lr1r17','EMOTIONS_Q9_Lr1r18','IMPACT_Q12_Lr1r1','IMPACT_Q12_Lr1r2','IMPACT_Q12_Lr1r3','IMPACT_Q12_Lr1r4','POST_FAVORABILITY_Q13_Lr1r1','POST_FAVORABILITY_Q13_Lr1r2','POST_FAVORABILITY_Q13_Lr1r3','POST_FAVORABILITY_Q13_Lr1r4','FIT_Q14_Lr1r1','FIT_Q14_Lr1r2','FIT_Q14_Lr1r3','FIT_Q14_Lr1r4','Q10_1_ConceptA','Q10_2_ConceptA','Q10_3_ConceptA','Q10_4_ConceptA','Q10_5_ConceptA','Q10_6_ConceptA','Q10_7_ConceptA','Q10_8_ConceptA','BELIEVABILITY_Q8_Lr1']
colms = Sec1

df_Label = pd.melt(All_Label, id_vars =['record'], value_vars = colms,var_name ='Question ID', value_name ='Text Responses')
df_Code = pd.melt(All_Code, id_vars =['record'], value_vars = colms,var_name ='Question ID', value_name ='Numeric Responses')

In [9]:
All_QH = pd.read_csv("1_CookieCutter\\Data\\All_QH.csv",encoding='cp1252')

inner_join_df = df_Label.merge(df_Code, how="inner", on=['record','Question ID'])
output_df = inner_join_df.merge(All_QH, how="inner", on=['Question ID'])

In [10]:
bool_series_1 = pd.notnull(output_df['Numeric Responses'])
bool_series_2 = pd.notnull(output_df['Text Responses'])

# displayed data only with team = NaN
output_df = output_df[bool_series_1 | bool_series_2]

In [11]:
cond_1 = ((output_df['QType']=='Check all') & (output_df['Numeric Responses']==0))
output_df = output_df[~cond_1]

In [12]:
output_df = output_df.replace(to_replace =['Others, Please specify','Others (Please Specify)'],value ="Others")

# To SQL Database

In [13]:
engine = sal.create_engine('mssql+pyodbc://BASRV10\SQLEXPRESS/CookieCutter?driver=SQL Server?Trusted_Connection=yes')
# establishing the connection to the databse using engine as an interface
conn = engine.connect()

In [14]:
output_df.to_sql('All_w1', con=engine, if_exists='append',index=False)